In [21]:
coviddata = pd.read_csv('/Users/katie/Downloads/covid_deaths_usafacts.csv')
coviddata = coviddata.drop(columns=['stateFIPS','countyFIPS'])
coviddeathsdata = coviddata.loc[coviddata['County Name'].str.contains('Statewide', case=False),:]
coviddata_meta = coviddeathsdata.loc[:, ['State']]
coviddeathsdata = coviddeathsdata.drop(columns=['County Name','State'])
fig = go.Figure(data=[go.Surface(z=coviddeathsdata.values)])
fig.update_layout(title='COVID-19 Deaths Data',autosize=True)
fig.show()

In [22]:
coviddata_scaled = coviddeathsdata-coviddeathsdata.mean()
u, s, v = np.linalg.svd(coviddata_scaled, full_matrices=True)
var_explained = np.round(s**2/np.sum(s**2), decimals=3)
labels= ['SV'+str(i) for i in range(1,51)]
svd_df = pd.DataFrame(u[:,0:50], index=coviddata_meta["State"].tolist(), columns=labels)
svd_df=svd_df.reset_index()
svd_df.rename(columns={'index':'State'}, inplace=True)
s = np.diag(s)
approx_uptorank1_meancentered = u[:,:1] @ s[0:1,:1] @ v[:1,:]
a = coviddeathsdata.mean()
b = a.values
approx_uptorank1 = approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_uptorank1)])
fig.update_layout(title='SV1 Only',autosize=True)
fig.show()

In [23]:
approx_uptorank2_meancentered = u[:,:2] @ s[0:2,:2] @ v[:2,:]
approx_justrank2 = approx_uptorank2_meancentered - approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_justrank2)])
fig.update_layout(title='SV2 Only',autosize=True)
fig.show()

In [24]:
approx_uptorank3_meancentered = u[:,:3] @ s[0:3,:3] @ v[:3,:]
approx_justrank3 = approx_uptorank3_meancentered - approx_uptorank2_meancentered - approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_justrank3)])
fig.update_layout(title='SV2 Only',autosize=True)
fig.show()

In [25]:
approx_uptorank3 = approx_uptorank3_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_uptorank3)])
fig.update_layout(title='Approximation Up To SV3',autosize=True)
fig.show()

In [26]:
residualnumbers = covid - approx_uptorank3
fig = go.Figure(data=[go.Surface(z=residualnumbers)])
fig.update_layout(title='Residual',autosize=True)
fig.show()